Data comes from the [WHO's interactive dashboard](https://www.who.int/data/gho/data/indicators/indicator-details/GHO/number-of-deaths-due-to-hiv-aids). The selected indicator is "Number of people dying from HIV-related causes", and the dataset holds values for each country from 2000-2023.

In [2]:
import pandas as pd

df = pd.read_csv('../../data/raw_data/WHO_HIV_deaths.csv')

In [3]:
df.columns

Index(['IndicatorCode', 'Indicator', 'ValueType', 'ParentLocationCode',
       'ParentLocation', 'Location type', 'SpatialDimValueCode', 'Location',
       'Period type', 'Period', 'IsLatestYear', 'Dim1 type', 'Dim1',
       'Dim1ValueCode', 'Dim2 type', 'Dim2', 'Dim2ValueCode', 'Dim3 type',
       'Dim3', 'Dim3ValueCode', 'DataSourceDimValueCode', 'DataSource',
       'FactValueNumericPrefix', 'FactValueNumeric', 'FactValueUoM',
       'FactValueNumericLowPrefix', 'FactValueNumericLow',
       'FactValueNumericHighPrefix', 'FactValueNumericHigh', 'Value',
       'FactValueTranslationID', 'FactComments', 'Language', 'DateModified'],
      dtype='object')

In [4]:
# Cell 1: Import libraries
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio
from IPython.display import HTML, display

In [5]:
# Cell 2: Define constants and helper functions

# Define PEPFAR countries
pepfar_countries = [
    "Botswana", "Côte d'Ivoire", "Ethiopia", "Kenya", "Mozambique", 
    "Namibia", "Nigeria", "Rwanda", "South Africa", "Tanzania", 
    "Uganda", "Zambia", "Guyana", "Haiti", "Viet Nam"
]

# Function to load and prepare data from CSV file
def load_hiv_data(file_path):
    """Load HIV deaths data from the WHO dataset CSV file."""
    df = pd.read_csv(file_path)
    
    # Keep only the columns we need
    df = df[['Location', 'Period', 'FactValueNumericHigh']]
    
    # Filter for PEPFAR countries and years between 2000-2023
    df = df[(df['Location'].isin(pepfar_countries)) & 
            (df['Period'] >= 2000) & (df['Period'] <= 2023)]
    
    return df

# Function to generate sample data for testing (when actual data isn't available)
def generate_sample_data():
    """Generate sample data with realistic patterns."""
    # Create an empty DataFrame
    years = list(range(2000, 2024))
    data = []
    
    for country in pepfar_countries:
        # Start with a random baseline
        base_deaths = 40000 + (pepfar_countries.index(country) * 5000)
        
        for year in years:
            deaths = base_deaths
            
            if year < 2004:
                # Before PEPFAR: slight increase or stable
                deaths *= (1 + (0.05 * (year - 2000) / 4))
            else:
                # After PEPFAR: gradual decrease
                years_since_pepfar = year - 2004
                decline_rate = 0.08 if year < 2010 else 0.05
                deaths *= (1 - decline_rate * (years_since_pepfar / 20))
            
            # Add some random variation
            noise = np.random.normal(1, 0.05)
            deaths = int(deaths * noise)
            
            data.append({
                'Location': country,
                'Period': year,
                'FactValueNumericHigh': deaths
            })
    
    return pd.DataFrame(data)

## Attempt #1

## PICK UP HERE: 

- finish making the hovered line highlighted in a bolder color to distinguish better 
- make filter option to see ALL countries vs PEPFAR coun

FINAL, perfectly fnunctioning one!!!

In [8]:
def create_final_hiv_visualization(df, filename="hiv_visualization_final.html"):
    import json
    
    # Get years and sort them
    years = sorted(df['Period'].unique())
    
    # PEPFAR countries (AS OF 2004)
    pepfar_countries = [
        "Botswana", "Côte d'Ivoire", "Ethiopia", "Kenya", "Mozambique", 
        "Namibia", "Nigeria", "Rwanda", "South Africa", "Tanzania", 
        "Uganda", "Zambia", "Guyana", "Haiti"
    ]
    
    # regions for coloring
    regions = {
        'Africa': ["Botswana", "Côte d'Ivoire", "Ethiopia", "Kenya", "Mozambique", 
                  "Namibia", "Nigeria", "Rwanda", "South Africa", "Tanzania", 
                  "Uganda", "Zambia"],
        'Caribbean': ["Guyana", "Haiti"]
    }
    
    # Assign colors by region
    region_colors = {
        'Africa': 'rgba(255, 99, 71, 0.7)',    # salmon 
        'Caribbean': 'rgba(135, 206, 250, 0.7)'  # Light blue
    }
    
    # highlight colors (brighter versions)
    highlight_colors = {
        'Africa': 'rgba(255, 99, 71, 1.0)',
        'Caribbean': 'rgba(135, 206, 250, 1.0)',
    }
    
    # traces for all countries
    all_traces = []
    all_countries = sorted(df['Location'].unique())
    
    # remove countries with only 100 deaths for every year
    countries_with_variation = []
    for country in all_countries:
        country_data = df[df['Location'] == country]
        if not country_data.empty:
            values = country_data['FactValueNumericHigh'].values
            if len(values) > 0 and not all(v == 100 for v in values):
                countries_with_variation.append(country)
    
    # 1) add non-PEPFAR countries (in white)
    non_pepfar_countries = [c for c in countries_with_variation if c not in pepfar_countries]
    
    for country in non_pepfar_countries:
        country_data = df[df['Location'] == country]

        if not country_data.empty:
            # Sort by year
            country_data = country_data.sort_values('Period')

            trace = {
                'x': country_data['Period'].tolist(),
                'y': country_data['FactValueNumericHigh'].tolist(),
                'mode': 'lines+markers',
                'name': country,
                'line': {
                    'color': 'rgba(255, 255, 255, 0.3)',  
                    'width': 1
                },
                'marker': {
                    'size': 4
                },
                # details-on-demand customed hover info
                'hovertemplate': (
                    '<span style="font-weight: bold; font-size: 16px; color: rgba(255, 255, 255, 1.0);">%{fullData.name}</span><br>' +
                    '<span style="color: white;">Year = %{x}</span><br>' +
                    '<span style="color: white;">Deaths = %{y:,}</span>' +
                    '<extra></extra>' 
                ),
                'hoverlabel': {
                    'bgcolor': '#121212',
                    'bordercolor': 'white',
                    'font': {
                        'family': 'Arial, sans-serif',
                        'size': 14
                    }
                },
                'visible': False  
            }
            
            all_traces.append(trace)
    
    # 2.) add PEPFAR countries
    for country in pepfar_countries:
        country_data = df[df['Location'] == country]

        if not country_data.empty:
            # Sort by year
            country_data = country_data.sort_values('Period')
            
            # Determine region
            region = 'Caribbean' if country in regions['Caribbean'] else 'Africa'
            color = region_colors[region]
            highlight_color = highlight_colors[region]
            
            # Create trace for this PEPFAR country
            trace = {
                'x': country_data['Period'].tolist(),
                'y': country_data['FactValueNumericHigh'].tolist(),
                'mode': 'lines+markers',
                'name': country,
                'line': {
                    'color': color,
                    'width': 2
                },
                'marker': {
                    'size': 6
                },
                # hover info
                'hovertemplate': (
                    '<span style="font-weight: bold; font-size: 16px; color:' + highlight_color + ';">%{fullData.name}</span><br>' +
                    '<span style="color: white;">Year = %{x}</span><br>' +
                    '<span style="color: white;">Deaths = %{y:,}</span>' +
                    '<extra></extra>'  
                ),
                'hoverlabel': {
                    'bgcolor': '#121212',
                    'bordercolor': 'white',
                    'font': {
                        'family': 'Arial, sans-serif',
                        'size': 14
                    }
                },
                'visible': True  
            }
            
            all_traces.append(trace)
    
    # aggregate data for all PEPFAR countries
    pepfar_data = df[df['Location'].isin(pepfar_countries)]
    aggregate_data = pepfar_data.groupby('Period')['FactValueNumericHigh'].sum().reset_index()
    
    # Add aggregate line (hidden by default)
    total_trace = {
        'x': aggregate_data['Period'].tolist(),
        'y': aggregate_data['FactValueNumericHigh'].tolist(),
        'mode': 'lines+markers',
        'name': 'Total',
        'line': {
            'color': 'white',
            'width': 3
        },
        'marker': {
            'size': 8
        },
        # Custom hover template for total
        'hovertemplate': (
            '<span style="font-weight: bold; font-size: 16px; color: white;">%{fullData.name}</span><br>' +
            '<span style="color: white;">Year = %{x}</span><br>' +
            '<span style="color: white;">Deaths = %{y:,}</span>' +
            '<extra></extra>'
        ),
        # Custom hover label styling
        'hoverlabel': {
            'bgcolor': '#121212',
            'bordercolor': 'white',
            'font': {
                'family': 'Arial, sans-serif',
                'size': 14
            }
        },
        'visible': 'legendonly'  # Visible only when selected from legend
    }
    
    all_traces.append(total_trace)
    
    # Calculate y-max for the annotation
    y_max = df['FactValueNumericHigh'].max()
    
    # Start building the HTML
    html_template = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>HIV Deaths Interactive Visualization</title>
    <!-- Import Plotly.js from CDN -->
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #111;
            color: white;
        }}
        #plot {{
            width: 100%;
            height: 700px;
        }}
        .container {{
            max-width: 1200px;
            margin: 0 auto;
        }}
        h1 {{
            text-align: center;
            margin-bottom: 30px;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1>HIV-Related Deaths in PEPFAR Countries (2000-2023)</h1>
        <div id="plot"></div>
    </div>

    <script>
        // Actual data from your DataFrame
        const data = {json.dumps(all_traces)};

        // Layout configuration
        const layout = {{
            hovermode: 'closest',
            paper_bgcolor: '#121212',
            plot_bgcolor: '#121212',  // Changed from rgba(0,0,0,0.8) to #121212
            font: {{
                color: 'white'
            }},
            xaxis: {{
                title: {{
                    text: 'Year',
                    font: {{
                        size: 16,
                        color: 'white'
                    }}
                }},
                gridcolor: 'rgba(255,255,255,0.1)',
                zerolinecolor: 'rgba(255,255,255,0.1)',
                color: 'white'
            }},
            yaxis: {{
                title: {{
                    text: 'HIV-Related Deaths',
                    font: {{
                        size: 16,
                        color: 'white'
                    }}
                }},
                gridcolor: 'rgba(255,255,255,0.1)',
                zerolinecolor: 'rgba(255,255,255,0.1)',
                // Use comma separator for thousands
                tickformat: ',',
                color: 'white',
                autorange: true
            }},
            legend: {{
                title: {{
                    text: 'Countries (click to show/hide)',
                    font: {{
                        color: 'white'
                    }}
                }},
                font: {{
                    color: 'white'
                }},
                orientation: "v",
                yanchor: "top",
                y: 1.0,
                xanchor: "left",
                x: 1.02,
                bgcolor: "rgba(0,0,0,0.5)",
                bordercolor: "rgba(255,255,255,0.3)",
                itemsizing: "constant",
                itemclick: "toggle",
                itemdoubleclick: "toggleothers",
                tracegroupgap: 5,
                valign: "top"
            }},
            shapes: [
                {{
                    // Vertical line for PEPFAR implementation 
                    // Extended to go all the way from top to bottom
                    type: 'line',
                    x0: 2004,
                    y0: -0.05, // Extend slightly below the bottom
                    x1: 2004,
                    y1: 1.05,  // Extend slightly above the top
                    yref: 'paper', // Reference to the entire paper coordinates
                    line: {{
                        color: "red",
                        width: 2,
                        dash: "dash"
                    }}
                }}
            ],
            annotations: [
                {{
                    // Move PEPFAR annotation to the top of the chart
                    x: 2004,
                    y: 1.05,
                    yref: 'paper',
                    text: "PEPFAR Implementation (2004)",
                    showarrow: true,
                    arrowhead: 1,
                    ax: 0,
                    ay: 30, // Point down from the top
                    font: {{
                        color: "red",
                        size: 12
                    }}
                }},
                {{
                    x: 0,
                    y: -0.15,
                    xref: "paper",
                    yref: "paper",
                    text: "Data Source: World Health Organization (WHO)",
                    showarrow: false,
                    font: {{
                        size: 10,
                        color: "white"
                    }},
                    align: "left"
                }}
            ],
            height: 700,
            margin: {{
                l: 90,
                r: 150,
                t: 100,
                b: 100
            }}
        }};

        // Create the plot
        Plotly.newPlot('plot', data, layout);

        // Store original line widths
        const originalLineWidths = data.map(trace => trace.line.width);
        
        // Add hover event to make line thicker
        document.getElementById('plot').on('plotly_hover', function(eventData) {{
            if (eventData.points && eventData.points.length > 0) {{
                const traceIndex = eventData.points[0].curveNumber;
                
                // Create new line widths array - thick for hovered, original for others
                const newWidths = originalLineWidths.map((width, index) => 
                    index === traceIndex ? width * 3 : width);
                
                // Update all traces with new line widths
                Plotly.restyle('plot', {{'line.width': newWidths}});
            }}
        }});

        // Add unhover event to reset line width
        document.getElementById('plot').on('plotly_unhover', function() {{
            // Reset all lines to original width
            Plotly.restyle('plot', {{'line.width': originalLineWidths}});
        }});

        // Get the number of non-PEPFAR countries
        const numNonPepfarCountries = {len(non_pepfar_countries)};
        const numPepfarCountries = {len(pepfar_countries)};
        
        // Add buttons to switch between PEPFAR only and ALL countries
        // Moved to the top of the chart
        const updateMenus = [{{
            type: "buttons",
            direction: "right",
            active: 0,
            buttons: [
                {{
                    args: [{{
                        "visible": Array(numNonPepfarCountries).fill(false)
                            .concat(Array(numPepfarCountries).fill(true))
                            .concat(['legendonly'])
                    }}],
                    label: "PEPFAR Countries",
                    method: "update"
                }},
                {{
                    args: [{{
                        "visible": Array(numNonPepfarCountries).fill(true)
                            .concat(Array(numPepfarCountries).fill(true))
                            .concat(['legendonly'])
                    }}],
                    label: "ALL Countries",
                    method: "update"
                }}
            ],
            pad: {{"r": 10, "t": 10}},
            showactive: true,
            x: 0.5,
            xanchor: "center",
            y: 1.1,   // Moved to the top of the chart area
            yanchor: "top",
            font: {{color: 'black'}},
            bgcolor: 'white',
            bordercolor: '#FFFFFF'
        }}];
        
        Plotly.relayout('plot', {{'updatemenus': updateMenus}});
    </script>
</body>
</html>
    """
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html_template)
    
    print(f"Final HIV visualization saved to {filename}")
    return filename

# Usage with actual data:
create_final_hiv_visualization(df, "../../interactive_viz_outputs/global_aids_deaths_linechart.html")

Final HIV visualization saved to ../../interactive_viz_outputs/global_aids_deaths_linechart.html


'../../interactive_viz_outputs/global_aids_deaths_linechart.html'